<a href="https://colab.research.google.com/github/yoohw0317/MILAB/blob/main/Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [13]:
torch.manual_seed
torch.set_printoptions(sci_mode=False, precision=20)

In [17]:
z = torch.FloatTensor([101, 102, 105, 109, 117, 118])

In [18]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([    0.00000003026258355021,     0.00000008226223258134,
            0.00000165228118476080,     0.00009021149162435904,
            0.26891666650772094727,     0.73099130392074584961])


In [5]:
hypothesis.sum()

tensor(1.)

# **Cross Entropy Loss**

In [19]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis) #prediction (y_hat)

tensor([[0.21000182628631591797, 0.22901710867881774902, 0.14481103420257568359,
         0.29076102375984191895, 0.12540905177593231201],
        [0.29387834668159484863, 0.28821721673011779785, 0.14599831402301788330,
         0.11258735507726669312, 0.15931883454322814941],
        [0.13891325891017913818, 0.17843909561634063721, 0.12999331951141357422,
         0.33737406134605407715, 0.21528023481369018555]],
       grad_fn=<SoftmaxBackward0>)


In [20]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([4, 4, 0])


In [21]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.]])

In [22]:
cost = (y_one_hot * - torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.96230924129486083984, grad_fn=<MeanBackward0>)


In [23]:
torch.log(F.softmax(z, dim=1))

tensor([[-1.56063902378082275391, -1.47395861148834228516,
         -1.93232560157775878906, -1.23525357246398925781,
         -2.07617449760437011719],
        [-1.22458934783935546875, -1.24404084682464599609,
         -1.92416024208068847656, -2.18402576446533203125,
         -1.83684778213500976562],
        [-1.97390556335449218750, -1.72350788116455078125,
         -2.04027223587036132812, -1.08656299114227294922,
         -1.53581464290618896484]], grad_fn=<LogBackward0>)

In [24]:
F.log_softmax(z, dim=1)

tensor([[-1.56063902378082275391, -1.47395861148834228516,
         -1.93232560157775878906, -1.23525357246398925781,
         -2.07617449760437011719],
        [-1.22458934783935546875, -1.24404072761535644531,
         -1.92416012287139892578, -2.18402576446533203125,
         -1.83684778213500976562],
        [-1.97390556335449218750, -1.72350788116455078125,
         -2.04027223587036132812, -1.08656299114227294922,
         -1.53581476211547851562]], grad_fn=<LogSoftmaxBackward0>)

In [25]:
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.96230924129486083984, grad_fn=<MeanBackward0>)

In [26]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.96230924129486083984, grad_fn=<NllLossBackward0>)

In [27]:
F.cross_entropy(z, y)

tensor(1.96230924129486083984, grad_fn=<NllLossBackward0>)

In [42]:
x_train = [[1, 2, 4, 6],
           [2, 4, 6, 3],
           [4, 2, 9, 8],
           [12, 24, 25, 28],
           [41, 90, 50, 40],
           [10, 20, 30, 45],
           [20, 100, 80, 77]]
y_train = [0, 3, 2, 2, 0, 3, 3]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [52]:
W = torch.zeros((4, 4), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()


    if epoch % 100 == 0:
      print(f'Epoch {epoch:4d}/{nb_epochs}, Cost: {cost.item():.6f}')

Epoch    0/1000, Cost: 1.386294
Epoch  100/1000, Cost: 1.227364
Epoch  200/1000, Cost: 1.218031
Epoch  300/1000, Cost: 1.204068
Epoch  400/1000, Cost: 1.196014
Epoch  500/1000, Cost: 1.190228
Epoch  600/1000, Cost: 1.186364
Epoch  700/1000, Cost: 1.183378
Epoch  800/1000, Cost: 1.180963
Epoch  900/1000, Cost: 1.178980
Epoch 1000/1000, Cost: 1.177330


In [53]:
W = torch.zeros((4, 4), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()


    if epoch % 100 == 0:
      print(f'Epoch {epoch:4d}/{nb_epochs}, Cost: {cost.item():.6f}')

Epoch    0/1000, Cost: 1.386294
Epoch  100/1000, Cost: 4.781619
Epoch  200/1000, Cost: 1.763522
Epoch  300/1000, Cost: 8.481661
Epoch  400/1000, Cost: 1.565037
Epoch  500/1000, Cost: 1.831817
Epoch  600/1000, Cost: 1.445968
Epoch  700/1000, Cost: 1.800390
Epoch  800/1000, Cost: 1.433392
Epoch  900/1000, Cost: 3.923111
Epoch 1000/1000, Cost: 4.121965


In [59]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 4)

    def forward(self, x):
        return self.linear(x)

In [60]:
model = SoftmaxClassifierModel()

In [61]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
      print(f'Epoch {epoch:4d}/{nb_epochs}, Cost: {cost.item():.6f}')

Epoch    0/1000, Cost: 23.283094
Epoch  100/1000, Cost: 3.000828
Epoch  200/1000, Cost: 20.661602
Epoch  300/1000, Cost: 12.299396
Epoch  400/1000, Cost: 0.592501
Epoch  500/1000, Cost: 4.038273
Epoch  600/1000, Cost: 0.025027
Epoch  700/1000, Cost: 0.013583
Epoch  800/1000, Cost: 0.010057
Epoch  900/1000, Cost: 0.008170
Epoch 1000/1000, Cost: 0.006942
